In [1]:
import pandas as pd 
import pandas_ta as ta 
import yfinance as yf 
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.io as pio

In [51]:
def chart(df):

    fig = go.Figure(
        data=[go.Candlestick(
            x=df['date'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close']
        )]
    )

    fig.update_layout(
        xaxis_rangeslider_visible = False
    )
    
    pio.show(fig, renderer='browser') 

In [52]:
def cleaning(df):
    df = df.copy()  # Make a copy to avoid modifying the original DataFrame
    df.columns = [name[0].lower() for name in df.columns]  # Convert column names to lowercase
    df.index = df.index.rename('date')  # Rename index to 'date'
    df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d %H:%M:%S')  # Format index as datetime string
    df = df.reset_index(drop=False)
    return df

In [64]:
def d1_fibonacci(df):

    df['date'] = pd.to_datetime(df['date']).dt.date
    daily_df = df.groupby('date').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })

    daily_df['fib_61_8'] = daily_df['high'] - (daily_df['high'] - daily_df['low']) * 0.618
    
    daily_df = daily_df.reset_index()

    df['fib_61_8'] = 0
    df[['date','fib_61_8']].merge(daily_df[['date','fib_61_8']])

    return daily_df, df

In [53]:
gold_15 = cleaning(yf.download('GC=F',period='8d', interval='15m'))

[*********************100%***********************]  1 of 1 completed


In [54]:
gold_15.head(3).loc[:]

,date,close,high,low,open,volume
0,2025-01-20 00:00:00,2747.199951,2748.000000,2746.699951,2747.100098,183
1,2025-01-20 00:15:00,2748.199951,2748.600098,2747.100098,2747.100098,313
2,2025-01-20 00:30:00,2749.000000,2749.199951,2746.899902,2748.100098,582


In [68]:
d1_fibonacci(gold_15)[1]['fib_61_8']

C:\Users\USER\AppData\Local\Temp\ipykernel_3700\233379322.py:17: UserWarning:

You are merging on int and float columns where the float values are not equal to their int representation.



0      0
1      0
2      0
3      0
4      0
      ..
596    0
597    0
598    0
599    0
600    0
Name: fib_61_8, Length: 601, dtype: int64

In [56]:
chart(gold_15)